In [ ]:
import os
import torch


torchrun --nproc_per_node=4 --rdzv_backend=c10d --rdzv_endpoint=127.0.0.1:29502 train.py

In [1]:
from transformers import BertConfig, BertModel
model = BertModel.from_pretrained('bert-base-uncased')

/home/cxd/storage/envs/py310_rasa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [2]:
import os
import torch

# 设置代理（本地 HTTP 代理，一般是 127.0.0.1:7890）
os.environ["HTTP_PROXY"] = "http://127.0.0.1:7890"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"

In [9]:
from my_model.model_person_search import ALBEF
import torch


/home/cxd/storage/envs/py310_rasa/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/cxd/storage/envs/py310_rasa/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [10]:
from transformers import BertModel, BertTokenizer
import os

# 设置本地模型路径
local_model_path = "./bert-base-uncased"

# 检查是否已存在模型文件（config.json 和 pytorch_model.bin 是必须的）
required_files = ["config.json", "model.safetensors", "vocab.txt"]
is_model_ready = all(os.path.exists(os.path.join(local_model_path, f)) for f in required_files)

# 如果没有模型文件就下载并保存
if not is_model_ready:
    print("本地模型不存在或不完整，正在从 Hugging Face 下载...")
    model = BertModel.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    os.makedirs(local_model_path, exist_ok=True)
    model.save_pretrained(local_model_path)
    tokenizer.save_pretrained(local_model_path)
else:
    print("发现本地模型，直接加载。")

# 加载模型和 tokenizer（不论是下载的还是本地已有的）
model = BertModel.from_pretrained(local_model_path)
tokenizer = BertTokenizer.from_pretrained(local_model_path)

print("模型和 tokenizer 加载成功。")


发现本地模型，直接加载。
模型和 tokenizer 加载成功。


In [11]:
import argparse
import ruamel.yaml as YAML
parser = argparse.ArgumentParser()
parser.add_argument('--config', default='./configs/PS_cuhk_pedes.yaml')
parser.add_argument('--output_dir', default='output/cuhk-pedes')
parser.add_argument('--checkpoint', default='')
parser.add_argument('--resume', action='store_true')
parser.add_argument('--eval_mAP', action='store_true', help='whether to evaluate mAP')
parser.add_argument('--text_encoder', default='bert-base-uncased')
parser.add_argument('--evaluate', action='store_true')
parser.add_argument('--device', default='cuda')
parser.add_argument('--seed', default=42, type=int)
parser.add_argument('--world_size', default=1, type=int, help='number of distributed processes')
parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
parser.add_argument('--distributed', default=True, type=bool)
parser.add_argument('--batch_size', default=13, type=int)
parser.add_argument('--embed_dim', default=577, type=int)
# parser.add_argument('--checkpoint',default='data/ALBEF/ALBEF.pth' ,type=str)
args= parser.parse_args([])
yaml = YAML.YAML(typ='rt')
config = yaml.load(open(args.config, 'r'))

In [12]:
config['embed_dim']


256

In [13]:
model = ALBEF(config=config, text_encoder=local_model_path, tokenizer=tokenizer)


In [17]:
args.checkpoint

'data/ALBEF/ALBEF.pth'

In [15]:
# #用来载入中途打断的模型
# checkpoint = torch.load(args.checkpoint, map_location='cpu')
# state_dict = checkpoint['model']
# msg = model.load_state_dict(state_dict, strict=False)
# print('load checkpoint from %s' % args.checkpoint)
# print(msg)


In [18]:
from my_model.vit import interpolate_pos_embed

checkpoint = torch.load(args.checkpoint, map_location='cpu')
state_dict = checkpoint['model']
if False:
    optimizer.load_state_dict(checkpoint['optimizer'])
    lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
    start_epoch = checkpoint['epoch'] + 1
    best = checkpoint['best']
    best_epoch = checkpoint['best_epoch']
else:
    # reshape positional embedding to accomodate for image resolution change
    pos_embed_reshaped = interpolate_pos_embed(state_dict['visual_encoder.pos_embed'], model.visual_encoder)
    state_dict['visual_encoder.pos_embed'] = pos_embed_reshaped
    m_pos_embed_reshaped = interpolate_pos_embed(state_dict['visual_encoder_m.pos_embed'],
                                                    model.visual_encoder_m)
    state_dict['visual_encoder_m.pos_embed'] = m_pos_embed_reshaped
msg = model.load_state_dict(state_dict, strict=False)
print('load checkpoint from %s' % args.checkpoint)
print(msg)

reshape position embedding from 256 to 576
reshape position embedding from 256 to 576
load checkpoint from data/ALBEF/ALBEF.pth
_IncompatibleKeys(missing_keys=['idx_queue', 'prd_head.weight', 'prd_head.bias', 'mrtd_head.weight', 'mrtd_head.bias'], unexpected_keys=[])


/home/cxd/storage/envs/py310_rasa/lib/python3.10/site-packages/torch/nn/modules/module.py:2409: UserWarning: for text_encoder.cls.predictions.decoder.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/home/cxd/storage/envs/py310_rasa/lib/python3.10/site-packages/torch/nn/modules/module.py:2409: UserWarning: for text_encoder_m.cls.predictions.decoder.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(


In [19]:
model

ALBEF(
  (visual_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
     